## Introduction

Dynamic Mode Decomposition (DMD) is a method, first published by Peter Schmid in 2010, that models the evolution of a system over time [1]. While first developed as a means of modeling dynamical systems such as fluid flows and other physical states, its applications have been expanded. Analyzing training a Neural Network as a dynamical system has been done previously[2] [3]. This work, specifically, investigates the application of DMD in accelerating the training of Graph Neural Networks. While similar to traditional feedforward networks GNNs differ in ways that may alter the appropriateness of various optimization algorithms. Also central to this work is the use of coarsening techniques on the graph data.

## Literature Review

In preparing for this work I reviewed multiple papers to learn about DMD and its application to accelerating training in artificial neural networks. I also familiarized myself with GNNs using a variety of resources. TensorFlow’s Machine Learning Tech Talks on GNNs by Senior Research Scientist at DeepMind, Petar Veličković, was particularly nice and I would recommend it to anyone looking to inquire about GNNs. The main papers I reviewed were [2], [3], and [4].

*In the case of [3], which totals 110 pages, ‘read’ is used to mean the abstract plus skimming some other sections. 

## Implementation

### Model

The model extends the pytorch nn.Module class and is implemented using PyTorch Geometric convolutional layers. The model has two convolutional layers, one dropout layer, and a dense Linear layer. Since this is a classification problem, softmax activation is used as the final layer. The model takes in three hyperparamaters: the number of features that each vertex has, the size of the convolutional layer, and the number of classes.

In [9]:
class GNN(torch.nn.Module):

    def __init__(self,num_features, hidden, out_features):
        super(GNN,self).__init__()
        seed = np.random.randint(0,high=999999,dtype=int)
        torch.manual_seed(seed)
        self.conv1 = GCNConv(num_features,hidden)
        self.conv2 = GCNConv(hidden,hidden)
        self.out_features = out_features
        self.end_layer = nn.Linear(hidden, out_features)
    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.end_layer.reset_parameters()
    def forward(self,torchG):

        x, edge_index = torchG.x, torchG.edge_index
        x = x.float()

        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x,training=self.training)
        x = self.conv2(x,edge_index)
        x = F.relu(x)
        x = self.end_layer(x)

        return F.log_softmax(x, dim=1)

### Training

In [11]:
def train_level(model, TGG_D, optimizer, loss_fn, level, m=None, pred_step=None, r=2, epochs=20):
    """    
    model: model to be trained
    TGG_D: Torch Geometric Graph Dictionary. Each TGG represents a different level of coarsening.
    optimizer: torch.optim optimizer used for training.
    loss_fn: torch loss function.
    level: The level of coarsening to perform training on. 
    epochs: total number of epochs to perform training.
        following arguments are supplied if DMD is used:
    m: DMDstep is performed after m epochs
    pred_step: how many timesteps to step forward.
    r: number of modes computed in DMD"""
    accuracy = []
    model.train()
    if m is None:
        m = epochs+1
    data = TGG_D[level].to(device)
    data.x = F.normalize(data.x,p=1)
    
    weights = [ np.empty(np.append(1, param.shape)) for param in model.parameters()]
    for i,param in enumerate(model.parameters()):
        weights[i][0] = param.detach()


    for epoch in tqdm(range(1,epochs+1)):
        if epoch % m == 0:
            print("DMD")
            DMDstep(model, weights, r, pred_step, params=None)
            weights = [ np.empty(np.append(1, param.shape)) for param in model.parameters()]
        train_epoch(model, data, optimizer, loss_fn)
        for i,param  in enumerate(model.parameters()):
            weights[i] = np.append(weights[i], param.detach().reshape(np.append(1,weights[i].shape[1:]).tolist()),axis=0)

We can call train_level in a variety of fashions. The one that I have found to be the most successful is perform 2 iterations of train_level per level. 1 with DMD and then one without DMD. We then iterate through each level starting with the coarsest and back to the original graph. Because of computational limitations we cannot train for as many epochs on the original graph as the coarsen graphs. 

## Results

The results are promising but have not yet shown anything to the extent of which prior papers have shown with respect to accelerations in training times. When limiting training to 30 seconds per level and 4 levels of training with DMD achieved an average accuracy of 0.8508. Without using DMD accuracy was very slightly lower: 0.8488.

These results are very sensitive to hyperparameter tuning and I suspect there is some configuration that is well suited to a particular class of problems

## Data

All of the data used in this work came from the Torch Geometric Datasets class. The class contains a plethora of graph datasets. We focused on the ones designed for vertex classification. The datasets in particular we examined were from the Planetoid subclass. This dataset contains three graphs where vertecies represent papers and edges represent citations. The features associated with each dataset are a Word2vec embedding for words contained in the paper.

### r/Place

r/Place was a collaboratize conducted by reddit in 2017 and 2022. In the project online users could place a single pixel on a public canvas up to once every 10 minutes. The data was subsequently released by Reddit. This data can be used to realize a graph. One such way is for vertices to represent users, with edges linked users who collaborated on the same square. Alternatively, vertices can represent squares which are connected by users. Unfortunately I did not have time to implement anything with this data as it is a unsupervised learning problem and required additional work beyond the main scope of the project. However, I did perform data processing which can be found in place_data_processing.py

## Future Work

The first step is to conduct hyperparameter optimization. If r is selected too large DMD takes too long. If r is too small DMD provides by results. This is just one example. A good hueristic is needed for hyperparameter selection. Once this is all complete in Python this all needs to be implemented in c. This will allow greater parralelization and will be where we can really speed up the expensive matrix operations needed to perform DMD.

## References

1. 
SCHMID, P. (2010). Dynamic mode decomposition of numerical and experimental data. Journal of Fluid Mechanics, 656, 5-28. doi:10.1017/S0022112010001217

2. Tano, Mauricio E. and Portwood, Gavin D. and Ragusa, Jean C. (2020) Accelerating Training in Artificial Neural Networks with Dynamic Mode Decomposition

3. Brunton, S.L., Budisic, M., Kaiser, E., & Kutz, J.N. (2021). Modern Koopman Theory for Dynamical Systems. SIAM Rev., 64, 229-340.

4. Akshunna S. Dogra and William T. Redman. 2020. Optimizing neural networks via koopman operator theory. In Proceedings of the 34th International Conference on Neural Information Processing Systems (NIPS'20). Curran Associates Inc., Red Hook, NY, USA, Article 176, 2087–2097.

5. Machine Learning Tech Talks, Senior Research Scientist at DeepMind, Petar Veličković